In [19]:
import os
import pathlib
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
data_dir = pathlib.Path('{}/1_preprocessing_all_domain/output_temp'.format(parent_dir))
output_dir = pathlib.Path('{}/output'.format(current_dir))
pathlib.Path.mkdir(output_dir, mode=0o777, exist_ok=True)

file_list = os.listdir(data_dir)
file_list = [pathlib.Path(filename).with_suffix('').name for filename in file_list]
file_list = ['abnormal_nephrotoxicity']

In [20]:
file_list

['abnormal_nephrotoxicity']

In [33]:
import pandas as pd

concat_df = pd.read_csv('{}/{}.txt'.format(data_dir, file_list[0]), index_col=False)

def timeSeriesToTimeData(timeseries_df):
    ts_df = timeseries_df.sort_values(['subject_id', 'concept_date'], ascending=[True, True])
    td_df = ts_df.groupby('subject_id', as_index=False).max()
    return td_df

concat_df = timeSeriesToTimeData(concat_df)

concat_df['cohort_start_date'] = pd.to_datetime(concat_df['cohort_start_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
concat_df['first_abnormal_date'] = pd.to_datetime(concat_df['first_abnormal_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
concat_df['concept_date'] = pd.to_datetime(concat_df['concept_date'], format='%Y-%m-%d %H:%M:%S', errors='raise')
    
concat_df['duration'] = (concat_df['concept_date']-concat_df['cohort_start_date']).dt.days+1
concat_df = concat_df.drop(['subject_id', 'unique_id', 'cohort_start_date', 'concept_date', 'first_abnormal_date'], axis=1)
concat_df



,age,sex,label,zolpidem,metoclopramide,furosemide,sodium chloride,tramadol,morphine,oxycodone,...,Primary malignant neoplasm of respiratory tract,500 ML glucose 50 MG/ML Injection,100 ML glucose 50 MG/ML Injection,X-ray of chest anteroposterior view,X-ray of chest posteroanterior view,Sodium Bicarbonate 1 MG/MG,Flavin-Adenine Dinucleotide 10 MG/ML / Liver Extract 0.015 ML/ML Injectable Solution [CHOONGWAE ADELAVIN 9],100 ML Benzydamine 1.5 MG/ML Mouthwash,piperacillin 4000 MG / tazobactam 500 MG Injection,duration
0,0.470588,0,0,0,1,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,57
1,0.529412,1,0,0,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,44
2,0.661765,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,44
3,0.705882,1,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,0,1,1,49
4,0.867647,0,0,0,1,1,1,0,0,1,...,1,0,0,0,0,0,0,0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,0.500000,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,13
435,0.514706,1,0,0,0,1,1,1,0,0,...,0,1,1,1,1,0,1,0,1,9
436,0.764706,1,0,0,1,0,1,0,0,1,...,1,0,0,0,1,0,0,0,0,57
437,0.544118,1,0,0,0,1,1,1,1,0,...,1,0,1,1,1,0,1,0,1,28


In [37]:
def split_x_y_data(df, OBP) :
    import numpy as np
    import pandas as pd

    y_data = df['label'].T.reset_index(drop=True) #df['label'].T.drop_duplicates().T.reset_index(drop=True)
    y_data = np.array(y_data)
    y_data = y_data[0:len(y_data):OBP].reshape(-1, 1).astype(int)
    #print(len(y_data), file=_logfile_)

    x_df = df.drop('label', axis=1)

    # 2-d data to 3-d data
    timestamp = OBP 
    x_data = np.array(x_df)
    #x_data = x_data.reshape(-1, timestamp, x_data.shape[1]) # -1(sample), timestamp, column
    #x_data.shape, y_data.shape

    # get Column data
    new_col = x_df.columns
    print(x_data.shape, y_data.shape, len(new_col))
    return x_data, y_data, new_col

x_data, y_data, new_col = split_x_y_data(concat_df, OBP=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=1, stratify=y_data) 

(439, 45) (439, 1) 45


In [38]:
def split_x_y_data(df, OBP) :
    import numpy as np
    import pandas as pd

    y_data = df['label'].T.reset_index(drop=True) #df['label'].T.drop_duplicates().T.reset_index(drop=True)
    y_data = np.array(y_data)
    y_data = y_data[0:len(y_data):OBP].reshape(-1, 1).astype(int)
    #print(len(y_data), file=_logfile_)

    x_df = df.drop('label', axis=1)

    # 2-d data to 3-d data
    timestamp = OBP 
    x_data = np.array(x_df)
    #x_data = x_data.reshape(-1, timestamp, x_data.shape[1]) # -1(sample), timestamp, column
    #x_data.shape, y_data.shape

    # get Column data
    new_col = x_df.columns
    print(x_data.shape, y_data.shape, len(new_col))
    return x_data, y_data, new_col

x_data, y_data, new_col = split_x_y_data(concat_df, OBP=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=1, stratify=y_data) 

(439, 45) (439, 1) 45


In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))\
        
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred, pred_proba)
    
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=x_train, ftr_test=x_test, tgt_train=y_train, tgt_test=y_test)

# import seaborn as sns
# plt.figure(figsize=(8, 4))
# plt.xticks(range(0, 30000, 1000), rotation=60)
# sns.distplot(card_df['Amount'])

오차 행렬
[[130   1]
 [  1   0]]
정확도: 0.9848, 정밀도: 0.0000, 재현율: 0.0000,    F1: 0.0000, AUC:0.3817


D:\Utility\Python37\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Utility\Python37\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
import xgboost as xgb
dtrain = xgb.DMatrix(data=x_train , label=y_train) 
dtest = xgb.DMatrix(data=x_test , label=y_test)
params = { 'max_depth':3, 'eta': 0.1, 'objective':'binary:logistic', 'eval_metric':'logloss' }
num_rounds = 400
# train 데이터 셋은 ‘train’ , evaluation(test) 데이터 셋은 ‘eval’ 로 명기합니다. 
wlist = [(dtrain,'train'),(dtest,'eval')]
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달 
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, early_stopping_rounds=200, evals=wlist )
pred_probs = xgb_model.predict(dtest)

# 예측 확률이 0.5 보다 크면 1 , 그렇지 않으면 0 으로 예측값 결정하여 List 객체인 preds 에 저장
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
def get_clf_eval(y_test, pred=None, pred_proba=None):
    accuracy = accuracy_score(y_test , pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba) 
    # ROC-AUC print 추가
    print('정확도: {0:.4f} ROC_AUC{1:.4f}'.format(accuracy, roc_auc))
get_clf_eval(y_test , preds, pred_probs)

[0]	train-logloss:0.60225	eval-logloss:0.60418
[1]	train-logloss:0.52762	eval-logloss:0.53114
[2]	train-logloss:0.46527	eval-logloss:0.47064
[3]	train-logloss:0.41248	eval-logloss:0.41900
[4]	train-logloss:0.36731	eval-logloss:0.37527
[5]	train-logloss:0.32836	eval-logloss:0.33762
[6]	train-logloss:0.29455	eval-logloss:0.30499
[7]	train-logloss:0.26503	eval-logloss:0.27656
[8]	train-logloss:0.23914	eval-logloss:0.25168
[9]	train-logloss:0.21636	eval-logloss:0.22984
[10]	train-logloss:0.19621	eval-logloss:0.21019
[11]	train-logloss:0.17836	eval-logloss:0.19323
[12]	train-logloss:0.16250	eval-logloss:0.17785
[13]	train-logloss:0.14836	eval-logloss:0.16457
[14]	train-logloss:0.13570	eval-logloss:0.15228
[15]	train-logloss:0.12441	eval-logloss:0.14183
[16]	train-logloss:0.11427	eval-logloss:0.13208
[17]	train-logloss:0.10517	eval-logloss:0.12353
[18]	train-logloss:0.09699	eval-logloss:0.11580
[19]	train-logloss:0.08964	eval-logloss:0.10904
[20]	train-logloss:0.08302	eval-logloss:0.10291
[2